In [13]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras import regularizers

(X_train_full, y_train_full), (X_test, y_test) = datasets.cifar10.load_data()

print(X_train_full.shape)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

X_val, X_train = X_train_full[:20000] / 255.0, X_train_full[20000:] / 255.0
y_val, y_train = y_train_full[:20000], y_train_full[20000:]
X_test = X_test / 255.0

(50000, 32, 32, 3)


In [14]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, GlobalMaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

def build_resnet50(input_shape=(32, 32, 3), dropout_rate = 0.5, l2_reg = 0.001):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)

    x = GlobalMaxPooling2D()(base_model.output)
    x = Dense(256, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    return model

input_shape = (32, 32, 3)
dropout_rate = 0.5 #Best result based on trying different values 0.5
l2_reg = 0.001 #Best result based on trying different values 0.001

model = build_resnet50(input_shape=input_shape, dropout_rate=dropout_rate, l2_reg=l2_reg)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

In [15]:
tf.random.set_seed(1)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=64, callbacks=[callback])

Epoch 1/100


469/469 [==============================] - 37s 65ms/step - loss: 2.8349 - accuracy: 0.2462 - val_loss: 2.6393 - val_accuracy: 0.2167
Epoch 2/100
469/469 [==============================] - 28s 61ms/step - loss: 2.1438 - accuracy: 0.3650 - val_loss: 1.9942 - val_accuracy: 0.3700
Epoch 3/100
469/469 [==============================] - 28s 61ms/step - loss: 1.7534 - accuracy: 0.4471 - val_loss: 1.6603 - val_accuracy: 0.4550
Epoch 4/100
469/469 [==============================] - 28s 60ms/step - loss: 1.5095 - accuracy: 0.5181 - val_loss: 1.6456 - val_accuracy: 0.4728
Epoch 5/100
469/469 [==============================] - 28s 61ms/step - loss: 1.3694 - accuracy: 0.5630 - val_loss: 1.5962 - val_accuracy: 0.4739
Epoch 6/100
469/469 [==============================] - 28s 61ms/step - loss: 1.2692 - accuracy: 0.5898 - val_loss: 1.5442 - val_accuracy: 0.5002
Epoch 7/100
469/469 [==============================] - 28s 60ms/step - loss: 1.1954 - accuracy: 0.6214 - val_loss: 1.4550 - val_accuracy: 0.51

Once tuning is finished, evaluate on testing set.

In [17]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 5s 16ms/step - loss: 1.2006 - accuracy: 0.6740


[1.2005839347839355, 0.6740000247955322]

In [18]:
model.save('ResNet.h5')